# Creating inputs and accessing stored data

This example shows how the ASCOT5 data is manipulated.

Pretty much everything that you do with ASCOT5, is done via the `Ascot` object.
So let's begin by inititalizing one.

In [1]:
import numpy as np
from a5py import Ascot
a5 = Ascot()


The `Ascot` object has `data` attribute that manages the data.
To get an overview of what is stored, use the `show_contents()` method.

In [ ]:
a5.data.show_contents()

Inputs: [only active shown]
options   *no inputs*

bfield    *no inputs*

efield    *no inputs*

marker    *no inputs*

plasma    *no inputs*

neutral   *no inputs*

wall      *no inputs*

boozer    *no inputs*

mhd       *no inputs*

asigma    *no inputs*

nbi       *no inputs*


Simulations:
No simulation results.



Now let's create some inputs.
The inputs are created with various `create_x` methods found in `data`.
ASCOT5 can be applied to a wide variety of problems which is why there are several different implementations for the magnetic field input for example.
The documentation offers an overview of all the inputs to help you find the one that you would need.

For this tutorial, we generate the most basic magnetic field, `B_TC` (B-Trivial-Cartesian), which is just an uniform field in Cartesian basis.
The corresponding factory method is `create_btc`, whose docstring tells us what data is needed to create this particular input variant.

In [ ]:
help(a5.data.create_btc)

Help on method create_btc in module a5py.data.bfield.btc:

create_btc(bxyz: Union[List[Union[int, float]], numpy.ndarray, unyt.array.unyt_array, NoneType] = None, jacobian: Union[List[Union[int, float]], numpy.ndarray, unyt.array.unyt_array, NoneType] = None, axisrz: Optional[Tuple[float, float]] = None, rhoval: float | None = None, psival: Optional[float] = None, note: Optional[str] = None, activate: bool = False, dryrun: bool = False, store_hdf5: Optional[bool] = None) -> a5py.data.bfield.btc.B_TC method of a5py.data.Ascot5IO instance
    Create a magnetic field input which is defined on a Cartesian basis.
    
    This method creates a magnetic field input which is simple and mainly
    serves to validate the orbit-integrators.
    
    The input also defines the magnetic axis coordinates and the value of
    (normalized) poloidal flux, but these are meaningless in this case and
    are only provided because the code assumes the existence of some flux
    surfaces.
    
    Paramete

So we need to provide `bxyz`, `jacobian`, `axisrz`, and `rhoval` whereas `psival` is optional.
The remaining arguments `note`, `activate`, `dryrun`, and `store_hdf5` are common in all factory methods and we will go through them later.

Now let's provide the mandatory data and generate the input.

In [ ]:
a5.data.create_btc(
    bxyz=[1.0, 0.0, 0.0],     # Magnetic field has only x-component
    jacobian=np.zeros((3,3)), # The field is uniform so the Jacobian is zero
    axisrz=[1.0, 0.0],        # For this field, the axis location is arbitrary
    rhoval=1.0,               # For this field, the flux surface coordinate is arbitrary
    )

<B_TC(qid=3977650745, date=2025-07-15 14:19:06, variant=B_TC)>

We can now confirm that the input was created.

In [ ]:
a5.data.show_contents()

Inputs: [only active shown]
options   *no inputs*

bfield    B_TC      4124471621 2025-07-15 14:06:04 + 1 other(s)
          "TAG"
efield    *no inputs*

marker    *no inputs*

plasma    *no inputs*

neutral   *no inputs*

wall      *no inputs*

boozer    *no inputs*

mhd       *no inputs*

asigma    *no inputs*

nbi       *no inputs*


Simulations:
No simulation results.



ASCOT5 supports having multiple inputs of a same type (for e.g. parameter scans), so let's create another input to demonstrate how this is reflected on how the data is accessed.
This time we use `note` parameter to document our extra feature.

In [ ]:
a5.data.create_btc(
    bxyz=[1.0, 0.0, 0.0],
    jacobian=np.zeros((3,3)),
    axisrz=[1.0, 0.0],
    rhoval=1.0,
    note="Second magnetic field."
    )

<B_TC(qid=3978940339, date=2025-07-15 14:28:20, variant=B_TC)>

The `data.show_contents()` method only displays one input, but the input group (which in this case is `bfield`) also have a `show_contents()` method which displays detailed information of all inputs belonging to that group.

In [ ]:
a5.data.bfield.show_contents()

B_TC       3978940339 2025-07-15 14:28:20
SECOND
Second magnetic field.

B_TC       3977650745 2025-07-15 14:19:06
TAG_0
TAG

B_TC       4124471621 2025-07-15 14:06:04 [active]
TAG_1
TAG




Each dataset has a type (in this case `B_TC`), quasi-random unique identifier (QID) consisting of ten numbers, and date when it was created.
Each dataset also has a note where one can write down some details about this particular dataset, as we just did.

Note that one of the inputs, the one that was created first, has a label <span style="color:green">[active]</span> next to it.
The label means that this particular input was set as *active* and will be used as an input on the next simulation.
When a new input is created, it is not automatically set as active, unless you set `activate=True` in the `create_x` factory methody.

The factory methods can also be called without supplying *any* mandatory parameters (if some mandatory parameters are supplied but not all, an exception is raised).
In this case, a dummy input data is created.
Furthermore, one can set `dryrun=True` to return the data object without adding it to the `data`.
This is useful when one wants to modify or check the input first.
We demonstrate this by creating a simple electric field input.


In [ ]:
edata = a5.data.create_etc(dryrun=True)
print(f"The value of the electric field is: {edata.exyz}")

print("Since this was a dry run, no data was stored:")
a5.data.efield.show_contents()

print("Now let's store the electric field data:")
a5.data.create_etc(**edata.export())
a5.data.efield.show_contents()

The value of the electric field is: [0. 0. 0.] V/m
Since this was a dry run, no data was stored:
E_TC       1411713509 2025-07-15 14:57:51 [active]
TAG
TAG


Now let's store the electric field data:
E_TC       0014644783 2025-07-15 14:57:58
TAG_0
TAG

E_TC       1411713509 2025-07-15 14:57:51 [active]
TAG_1
TAG




Let us now create rest of the inputs and run a quick simulation.

In [ ]:
from a5py.templates import PremadeMagneticField

a5 = Ascot()
template = PremadeMagneticField(a5, field="iter-baseline")
template.create_input()

a5.data.create_options(
    enable_orbit_following=True,
    activate_simulation_time_limits=True,
    simulation_mode=1,
)

<Options(qid=2782212059, date=2025-08-30 17:06:12, variant=Options)>

The contents are now more complete.

In [3]:
a5.init(0)

AscotIOException: No active data. Perhaps this node is empty?

In [4]:
from pathlib import Path
from IPython.display import Markdown

code = Path("../examples/mpirun.py").read_text()
Markdown(f"```python\n{code}\n```")

```python
from a5py import Ascot
```

## Accessing data